In [1]:
import cv2, os
import numpy as np
from tqdm import tqdm
from PIL import Image
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
path = "EHF/"

data = []
for images in tqdm(os.listdir(path)):
    if images.endswith('.jpg'):
        image_path = path + images
        image = cv2.imread(image_path)
        image = cv2.resize(image,(256,256))
        data.append(image)

100%|███████████████████████████████████████████████████████████████████████████████| 604/604 [00:02<00:00, 239.97it/s]


In [3]:
def load_ply_data(ply_pat):
    pcd = o3d.io.read_point_cloud(ply_path)
    vertices = np.asarray(pcd.points)
    min_coords = np.min(vertices, axis=0)
    max_coords = np.max(vertices, axis=0)
    normalized_vertices = (vertices - min_coords) / (max_coords - min_coords)
    image_array = np.zeros((256, 256, 3), dtype=np.uint8)  # Adjust the image size as needed
    for vertex in normalized_vertices:
        x, y, z = vertex
        image_x = int(x * (image_array.shape[1] - 1))
        image_y = int((1 - y) * (image_array.shape[0] - 1))  # Reverse the y-coordinate
        image_array[image_y, image_x, :] = 255 
    image = Image.fromarray(image_array)
    image.save("check.jpg")
    img = cv2.imread("check.jpg")
    return img

In [4]:
meshes = []

for images in tqdm(os.listdir(path)):
    if images.endswith('.ply'):
        ply_path = path + images
        ply_data = load_ply_data(ply_path)
        meshes.append(ply_data)

100%|███████████████████████████████████████████████████████████████████████████████| 604/604 [00:04<00:00, 140.95it/s]


In [5]:
data = np.array(data)#, dtype="float32") / 255.0
meshes = np.array(meshes)

In [6]:
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, concatenate, LeakyReLU,Concatenate,Flatten,Dense
from tensorflow.keras.models import Model

In [14]:
# Define the generator model
def build_generator():
    input_shape =(256,256, 3)
    num_filters = 32

    # Define the input layers
    input_image = Input(shape=input_shape)

    # Encoder
    enc1 = Conv2D(num_filters, kernel_size=4, strides=2, padding='same', activation='relu')(input_image)
    enc2 = Conv2D(num_filters*2, kernel_size=4, strides=2, padding='same', activation='relu')(enc1)
    enc3 = Conv2D(num_filters*2, kernel_size=4, strides=2, padding='same', activation='relu')(enc2)
    enc4 = Conv2D(num_filters*4, kernel_size=4, strides=2, padding='same', activation='relu')(enc3)
    # Decoder
    dec4 = Conv2DTranspose(num_filters*4, kernel_size=4, strides=2, padding='same', activation='relu')(enc4)
    dec4 = concatenate([dec4, enc3], axis=-1)
    dec5 = Conv2DTranspose(num_filters*2, kernel_size=4, strides=2, padding='same', activation='relu')(dec4)
    dec5 = concatenate([dec5, enc2], axis=-1)
    dec6 = Conv2DTranspose(num_filters*2, kernel_size=4, strides=2, padding='same', activation='relu')(dec5)
    #dec6 = concatenate([dec6, enc1], axis=-1)
    #dec7 = Conv2DTranspose(num_filters, kernel_size=4, strides=2, padding='same', activation='relu')(dec6)

    # Output
    output_image = Conv2DTranspose(3, kernel_size=4, strides=2, padding='same', activation='tanh')(dec6)

    # Define the model inputs and outputs
    generator_inputs = input_image
    generator_outputs = output_image
    model = tf.keras.Model(generator_inputs, generator_outputs, name='generator')
    #model.summary()
    return model

In [18]:
def build_discriminator():
    input_shape = (256, 256, 3)
    input_a = Input(shape=input_shape)

    d = Conv2D(64, 4, strides=2, padding='same')(input_a)
    d = LeakyReLU(alpha=0.2)(d)

    d = Conv2D(128, 4, strides=2, padding='same')(d)
    d = LeakyReLU(alpha=0.2)(d)

    d = Conv2D(256, 4, strides=2, padding='same')(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Flatten()(d)
    
    output = Dense(1, activation='sigmoid')(d)
    model = Model(inputs=input_a, outputs=output)
    return model

In [19]:
import tensorflow as tf
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [20]:
generator = build_generator()
discriminator = build_discriminator()

In [21]:
adgan_input_image = Input(shape=(256, 256, 3))
adgan_input_mask = Input(shape=(256, 256, 3))
generated_image = generator(adgan_input_image)
discriminator.trainable = False
validity = discriminator([adgan_input_image, generated_image])
adgan = Model(inputs=adgan_input_image, outputs=validity)

In [22]:
generator.compile(loss='mae', optimizer=generator_optimizer)
discriminator.compile(loss='binary_crossentropy', optimizer=discriminator_optimizer)

In [23]:
@tf.function
def generate_outputs(image, mask):
    # Generate image using generator
    generated_image = generator([image, mask], training=False)
    # Evaluate discriminator outputs
    disc_real_output = discriminator([image, mask], training=False)
    disc_generated_output = discriminator([image, generated_image], training=False)
    return disc_generated_output, generated_image, disc_real_output

In [25]:
disc_gen_outputs, gen_images, disc_real_outputs = [], [], []
target=[]

for image, mask in zip(data, meshes):
    image = np.expand_dims(image,axis=0)
    mask = np.expand_dims(mask,axis=0)
    disc_gen_output, gen_image, disc_real_output = generate_outputs(image)
    disc_gen_outputs.append(disc_gen_output)
    gen_images.append(gen_image)
    disc_real_outputs.append(disc_real_output)
    target.append(image)
    
# Define your loss functions
def generator_loss(disc_gen_outputs, gen_images, target):
    l1_loss = tf.reduce_mean(tf.abs(target - gen_output))
    return l1_loss

def discriminator_loss(real_output=disc_real_outputs, generated_output=disc_gen_outputs):
    real_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(real_output), real_output)
    generated_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.zeros_like(generated_output), generated_output)
    total_disc_loss = real_loss + generated_loss
    return total_disc_loss

adgan.compile(optimizer=generator_optimizer, loss=generator_loss, metrics=['mae', 'accuracy'])
adgan.fit(data,meshes, epochs=100)

tf.keras.models.save_model(generator, 'generator.h5')
tf.keras.models.save_model(discriminator, 'discriminator.h5')

TypeError: in user code:


    TypeError: outer_factory.<locals>.inner_factory.<locals>.tf__generate_outputs() missing 1 required positional argument: 'mask'
